In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_excel('/content/Sample Data.xlsx', index_col=0)

# Extract the 'Year' and 'Keywords' columns
df = df[['Year', 'keywords']]

# Drop rows with NaN values in the 'Keywords' column
df = df.dropna(subset=['keywords'])

# Convert the keywords column to string type
df['keywords'] = df['keywords'].astype(str)

# Split the keywords column by '|' and create a list of keywords for each row
df['keywords'] = df['keywords'].str.split('|')

# Create a list of keywords to search for
keywords_to_search = ['deep learning','imaging', 'machine learning','convolutional neural network','computational pathology', 'digital pathology','VIDEOMICROSCOPY', '2D-barcode', 'digital imaging', 'image processing', 'neural network']
# Create a boolean mask for rows that contain the keywords
mask = df['keywords'].apply(lambda x: any(keyword in x for keyword in keywords_to_search))

# Apply the boolean mask to filter the rows
df_filtered = df[mask]

# Explode the keywords column to create separate rows for each keyword
df_filtered = df_filtered.explode('keywords')

# Group by year and keyword, and count the occurrences
result = df_filtered.groupby(['Year', 'keywords']).size().unstack(fill_value=0)

# Reset the index to make years as columns and keywords as rows
result = result.reset_index()

# Rename the columns for readability
result.columns.name = None

# Set the 'Year' column as the index
result.set_index('Year', inplace=True)

# Filter the resulting DataFrame to only include the keywords in keywords_to_search
result = result[result.columns.intersection(keywords_to_search)]
# Reset the index to make years as columns and keywords as rows
result = result.reset_index()
# Print the resulting DataFrame
print(result)



    Year  2D-barcode  VIDEOMICROSCOPY  computational pathology  \
0   1990           0                1                        0   
1   1991           0                2                        0   
2   1992           0                4                        0   
3   1993           0               12                        0   
4   1994           0                5                        0   
5   1995           0               13                        0   
6   1996           0                0                        0   
7   1997           0                0                        0   
8   1998           0                0                        0   
9   1999           0                0                        0   
10  2000           0                0                        0   
11  2002           0                0                        0   
12  2003           0                0                        0   
13  2004           0                0                        0   
14  2005  

In [2]:
# Save the filtered DataFrame as an Excel file
##result.to_excel('filtered_results.xlsx', index=False)

Instead of saving records and add merged columns, you can run the below code. 
Just remember to add every set of merged terms as a list such as ['ML','Machine Learning','machine learning']

p.s. note that this code is case sensitive (ML is not ml) so check the errors if occurder for checking them... 

In [3]:
Merged = [
        ['Deep Learning','Deep learning'],
        ['Machine Learning','machine learning']
         ]

def sum_reduced_keys(df,x):
    name = x[0]
    new = df[x].sum(axis=1)
    df.drop(columns=x,inplace=True)
    df[name]=new

for x in Merged: sum_reduced_keys(result,x)
    
new_df = result

KeyError: ignored

In [ ]:
##new_df = pd.read_excel('filtered_results.xlsx', index_col=0)

In [ ]:
import numpy as np
# Calculate CAGR for each keyword
result_cagr = pd.DataFrame(index=new_df.columns, columns=['CAGR'])
for keyword in new_df.columns:
    start_value = new_df[keyword].loc[new_df[keyword].ne(0).idxmax()]  # First non-zero value
    end_value = new_df[keyword].iloc[-1]  # Value in the last row
    n = new_df[keyword].ne(0).sum()  # Number of non-zero values
    cagr = (end_value / start_value)**(1 / n) - 1  # Calculate CAGR
    result_cagr.at[keyword, 'CAGR'] = cagr if cagr != np.inf else np.nan

# Calculate share of keywords
result_share = pd.DataFrame(index=df.columns, columns=['Share'])
for keyword in new_df.columns:
    last_two_years_sum = new_df[keyword].iloc[-2:].sum()  # Sum of values for last two years
    total_sum = new_df[keyword].sum()  # Sum of all years
    share = (last_two_years_sum / total_sum) * 100  # Calculate share as percentage
    result_share.at[keyword, 'Share'] = share

    # Calculate average of shares
avg_share = result_share['Share'].mean()

# Create a new column to store quartile information
result_share['Quartile'] = ''
result_cagr['Quartile'] = ''

# Loop through each keyword
for keyword in new_df.columns:
    share = result_share.at[keyword, 'Share']
    cagr = result_cagr.at[keyword, 'CAGR']

    if share > avg_share:
        if cagr >= 0:
            result_share.at[keyword, 'Quartile'] = 'Dominant Technologies'
            result_cagr.at[keyword, 'Quartile'] = 'Dominant Technologies'
        else:
            result_share.at[keyword, 'Quartile'] = 'Saturated Technologies'
            result_cagr.at[keyword, 'Quartile'] = 'Saturated Technologies'
    else:
        if cagr > 0:
            result_share.at[keyword, 'Quartile'] = 'Emerging Technologies'
            result_cagr.at[keyword, 'Quartile'] = 'Emerging Technologies'
        else:
            result_share.at[keyword, 'Quartile'] = 'Declining Technologies'
            result_cagr.at[keyword, 'Quartile'] = 'Declining Technologies'

# Calculate sum of frequency for node size
result_freq_sum = new_df.sum()

In [ ]:
# Print CAGR results
print("CAGR Results:")
print(result_cagr)

# Print Share results
print("Share Results:")
print(result_share)


In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axes
fig, ax = plt.subplots(figsize=(8, 6))  # Update figsize to desired width and height in inches

# Define colors for each quartile
colors = {
    'Emerging Technologies': 'green',
    'Dominant Technologies': 'blue',
    'Saturated Technologies': 'red',
    'Declining Technologies': 'gray'
}

# Loop through each keyword
for keyword in new_df.columns:
    share = result_share.at[keyword, 'Share']
    cagr = result_cagr.at[keyword, 'CAGR']
    freq_sum = result_freq_sum.at[keyword]

    # Get the quartile for the keyword
    quartile = result_cagr.at[keyword, 'Quartile']

    # Get the color and size based on the quartile and frequency sum
    color = colors[quartile]
    size = freq_sum 

    # Plot the keyword as a scatter point with label
    ax.scatter(share, cagr, c=color, s=size, label=keyword)

    # Add text label for each keyword
    ax.annotate(keyword, (share, cagr), textcoords="offset points", xytext=(0, 10), ha='center', va='bottom', fontsize=8)
    
# Calculate mean of x-axis and plot vertical line
mean_share = new_df.sum(axis=1).mean()
ax.axvline(mean_share, color='black', linestyle='--', label='Mean Share')

# Plot horizontal line for CAGR = 0
ax.axhline(0, color='gray', linestyle='--', label='CAGR = 0')

# Set labels and title
ax.set_xlabel('Share of Keywords')
ax.set_ylabel('CAGR')
ax.set_title('Keyword Analysis')


# Show the plot
plt.show()


In [ ]:
mean_share

In [ ]:
! pip install pymannkendall

In [ ]:
import pandas as pd
import matplotlib as plt
import pymannkendall as mk
import numpy as np

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Keyword', 'Test Statistic', 'p-value', 'Trend', 'h', 'z', 'Tau', 's', 'var_s', 'slope'])

# Loop through each keyword column in the DataFrame
for col in new_df.columns:
    # Perform Mann-Kendall test on the column
    result = mk.original_test(new_df[col].values)
    
    # Extract the test statistic, p-value, and trend (if any)
    test_statistic = result[0]
    p_value = result[1]
    trend = result[2]
    h = result[3]
    z = result[4]
    Tau = result[5]
    s = result[6]
    var_s = result[7]
    slope = result[8]

    
# Append the results to the DataFrame
    results_df = results_df.append({'Keyword': col, 'Test Statistic': test_statistic, 'p-value': p_value, 'Trend': trend, 'h': h, 'z': z, 'Tau': Tau, 's': s, 'var_s': var_s, 'slope': slope}, ignore_index=True)

# Print the results in tabular format
print(results_df)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Transpose the DataFrame to switch the years to x-axis and keywords to y-axis
df_transposed = new_df.T


# Set the plot size
plt.figure(figsize=(15, 6))

# Create a heatmap using seaborn
sns.heatmap(df_transposed, cmap='YlGnBu', cbar_kws={'label': 'Frequency'}, annot=True, fmt="d")

# Set the title and labels
plt.title('Keyword Frequencies Heatmap')
plt.xlabel('Keywords')
plt.ylabel('Year')

# Rotate x-axis labels for better visibility
plt.xticks(rotation=90)

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Set plot style for lines
line_styles = ['-', '--', '-.', ':']  # Specify different line styles
num_styles = len(line_styles)

# Set plot size
plt.figure(figsize=(10, 6))

# Loop through each column (keyword) in the DataFrame and plot a line
for i, column in enumerate(new_df.columns):
    # Get x-axis (years) and y-axis (frequencies) data
    x = new_df.index
    y = new_df[column]
    
    # Plot the line with the specified style and label
    plt.plot(x, y, linestyle=line_styles[i % num_styles], label=column)

# Set log scale for y-axis
plt.yscale('log')

# Set title and labels
plt.title('Keyword Frequencies over Years (Log Scale)')
plt.xlabel('Year')
plt.ylabel('Frequency (log scale)')

# Add legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import pandas as pd

def klinsberg_burst_detection(values, threshold=0.5, min_burst_length=2):
    """
    Detect bursts in a time series using the Kinsberg algorithm.
   
    Args:
        values (np.array): 1D numpy array of values.
        threshold (float): Threshold value for detecting bursts. Default is 3.
        min_burst_length (int): Minimum length of a burst to be considered. Default is 2.
   
    Returns:
        bursts (list): List of dictionaries containing 'Weight', 'Length', 'Start', and 'End' values for each burst.
    """
    bursts = []
    n = len(values)
    mean = np.mean(values)
    std = np.std(values)
   
    for i in range(n):
        if values[i] > mean + threshold * std:
            start = i
            end = i
            weight = values[i] - mean
            length = 1
           
            # Extend the burst
            for j in range(i+1, n):
                if values[j] > mean + threshold * std and j == end + 1:
                    weight += values[j] - mean
                    end = j
                    length += 1
                else:
                    break
           
            # Check if the burst meets the minimum length requirement
            if length >= min_burst_length:
                burst = {'Weight': weight, 'Length': length, 'Start': start, 'End': end}
                bursts.append(burst)
   
    return bursts

# Read the Excel file into a DataFrame
# new_df = pd.read_excel('filtered_results.xlsx', index_col=0)

# Get all column names except for the 'Year' column
keywords = new_df.columns.tolist()[1:]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Keyword', 'Weight', 'Length', 'Start', 'End'])

# Loop through each keyword and calculate bursts
for keyword in keywords:
    # Get the values for the current keyword
    values = new_df[keyword].values
   
    # Call the klinsberg_burst_detection function
    bursts = klinsberg_burst_detection(values)
   
    # Merge consecutive bursts within a certain threshold of years
    merged_bursts = []
    for burst in bursts:
        if len(merged_bursts) > 0 and burst['Start'] - merged_bursts[-1]['End'] <= 1:
            # If the current burst starts within 1 year of the previous burst ending, merge them
            merged_bursts[-1]['End'] = burst['End']
            merged_bursts[-1]['Length'] = burst['End'] - merged_bursts[-1]['Start'] + 1
            merged_bursts[-1]['Weight'] += burst['Weight']
        else:
            # Otherwise, add the burst as a new burst
            merged_bursts.append(burst)
   
    # Loop through each merged burst and append the results to the DataFrame
    for burst in merged_bursts:
        # Get the start and end indices from the burst dictionary
        start = new_df.index[burst['Start']]
        end = new_df.index[burst['End']]
       
        # Append the burst details to the DataFrame
        results_df = results_df.append({'Keyword': keyword,
                                        'Weight': burst['Weight'],
                                        'Length': burst['Length'],
                                        'Start': start,'End': end}, ignore_index=True)
#Sort the results by keyword, start date, and end date

results_df.sort_values(by=['Keyword', 'Start', 'End'], inplace=True)
#Reset the index of the results DataFrame

results_df.reset_index(drop=True, inplace=True)
#Print the final results

print(results_df)

In [ ]:
# Save the results DataFrame to a CSV file
results_df.to_csv('burst_results.csv', index=False)


In [ ]:
# Load the dataset from CSV
df3 = pd.read_csv('burst_results.csv')
